In [1]:
import os

In [2]:
!pwd
os.chdir("../")
!pwd

/home/tensorthiru/ml_lifecycle_sample/research
/home/tensorthiru/ml_lifecycle_sample


In [3]:
!pwd

/home/tensorthiru/ml_lifecycle_sample


In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path

In [5]:
from src.pipeline_sample.utils.common import *
from src.pipeline_sample.constants import *

In [6]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath: Path = CONFIG_FILE_PATH,
                 schema_file_path: Path = SCHEMA_FILE_PATH,
                 params_file_path: Path = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)
        
        if type(self.config.artifacts_root) == str:
            create_directories([self.config.artifacts_root])
        else:
            create_directories(self.config.artifacts_root)
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        
        config = self.config.data_ingestion
        return  DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=Path(config.local_data_file)
        )
        

In [7]:
import urllib.request as request
from src.pipeline_sample import logger

In [8]:
# First component
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        create_directories([self.config.root_dir])
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(self.config.source_URL, self.config.local_data_file)
            logger.info(f"Downloaded file from {self.config.source_URL} with informations: {headers} to {filename}")
        else:
            logger.info(f"File already exists at {self.config.local_data_file}")
            
            

In [9]:
try:
    config_manager = ConfigurationManager()
    data_ingestion_config = config_manager.get_data_ingestion_config()
    data_ingestion_component = DataIngestion(config=data_ingestion_config)
    data_ingestion_component.download_file()
except Exception as e:
    logger.error(f"Failed to download file with error: {e}")
    raise e

[2025-01-29 15:05:59,512: INFO: common: YAML file laoded successfully: config/config.yaml]
[2025-01-29 15:05:59,514: INFO: common: YAML file laoded successfully: params.yaml]
[2025-01-29 15:05:59,515: INFO: common: YAML file laoded successfully: schema.yaml]
[2025-01-29 15:05:59,518: INFO: common: Directory created: artifacts]
[2025-01-29 15:05:59,519: INFO: common: Directory created: artifacts/data_ingestion]
[2025-01-29 15:05:59,718: INFO: 2054374211: Downloaded file from https://raw.githubusercontent.com/mlflow/mlflow-example/refs/heads/master/wine-quality.csv with informations: Connection: close
Content-Length: 264426
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: text/plain; charset=utf-8
ETag: "b59f798118e63a6fe2399f503b5411c00a46fb511fc74357511ef218d79f7186"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: